In [1]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append('../')
from benchmarks.inference_models import *

models = (
        Banana(),
        Gaussian(ndims=100, eigenvalues='Gamma', numpy_seed= rng_inference_gym_icg),
        GermanCredit(),
        Brownian(),
        ItemResponseTheory(),
        StochasticVolatility()
)

dfs = [pd.read_csv('sequential_results/' + model.name + str(model.ndims) + '0.csv') for model in models]

round = lambda x: np.round(x, 3)
#round = lambda x: x

def get_mclmc(df, num_windows):
    x= df[(df['sampler'] == 'mclmc:st3') & (df['num_windows'] == num_windows)]['ess_avg'].item()
    if x == 0.:
        return np.inf
    else:
        return round(1./x)

def get_nuts(df):
    return round(1./df[df['sampler'] == 'nuts']['ess_avg'].item())

df = pd.DataFrame(np.array([[get_mclmc(df, 1) for df in dfs], [get_mclmc(df, 2) for df in dfs], [get_nuts(df) for df in dfs]]).T, columns= ['MCLMC1', 'MCLMC2', 'NUTS'])
df.insert(0, 'model', ['Banana', 'Ill Conditioned Gaussian', 'Sparse Logistic Regression', 'Brownian Motion', 'Item Response Theory', 'Stochastic Volatility'])

display(df)
df_old = pd.DataFrame([['Sparse Logistic Regression', 0.006, 0.0014], ['Stochastic Volatility', 0.011, 0.006]], columns = ['model', 'MCLMC1', 'NUTS'])
display(df_old)
#df = df.style.format(index=False, precision=3)
#print(df.to_latex(index= False))



No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


,model,MCLMC1,MCLMC2,NUTS
0,Banana,53.76,50.14,642.109
1,Ill Conditioned Gaussian,326.74,334.52,1139.065
2,Sparse Logistic Regression,41.36,41.94,103.974
3,Brownian Motion,20.58,16.90,53.837
4,Item Response Theory,15.96,16.12,73.245
5,Stochastic Volatility,inf,inf,299.192


,model,MCLMC1,NUTS
0,Sparse Logistic Regression,0.006,0.0014
1,Stochastic Volatility,0.011,0.0060


In [10]:
pf= pd.read_csv('submission/pathfinder_convergence.csv', sep= '\t')
pf_cost = pd.read_csv('submission/pathfinder_cost.csv', sep= '\t')

for model in models:
    grads = np.max(np.array(pf_cost[model.name]))
     
    bavg, bmax = pf[pf['name'] == model.name][['bavg', 'bmax']].to_numpy()[0]
    print(f'{model.name}: grads={int(grads)}, bmaxsq= {bmax}')
    

Banana: grads=67, bmaxsq= 0.022757987
Gaussian_Gamma_1: grads=81, bmaxsq= 7.1916775e+16
GermanCredit: grads=135, bmaxsq= 21.520971
Brownian: grads=88, bmaxsq= 2316206000.0
ItemResponseTheory: grads=77, bmaxsq= 536717560000.0
StochasticVolatility: grads=80, bmaxsq= inf
